### 목표
    - 타이타닉 데이터를 활용하여 생존자/사망자 예측해보자
    - kaggle 경진대회에 참가하여 우리 점수를 확인해보자
    - 머신러닝의 전체 과정을 체험해보자
    
#### 머신러닝 과정
    - 1. 문제정의 : 목표설정, 어떤 모델
    - 2. 데이터 수집 : 분류를 할거라면 class로 담긴 레이블이 있는 데이터 수집 같은
    - 3. 데이터 전처리 : 이상치/결측치 처리, 특성 처리(특성공학)
    - 4. 데이터 탐색(탐색적 데이터 분석): 기술 통계, 특성 과의 관계
    - 5. 모델 선택 및 하이퍼파리미터 설정
    - 6. 모델 선택 및 평가
    - 7. 모델 서비스화
    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # 시각화 라이브러리
import seaborn as sns # 시각화 라이브러리
# train, test 분리하는 도구
from sklearn.model_selection import train_test_split
# 평가 도구
from sklearn.metrics import accuracy_score
# tree model - 사망/생존 이진분류
from sklearn.tree import DecisionTreeClassifier

 0   Survived  - 생존/ 사망 여부 (답 데이터, Class Label)  
 1   Pclass    - 승객의 등급(1- 귀족층,2- 중간층,3- 하위계층)  
 2   Name      - 이름        
 3   Sex       - 성별(male, female)
 4   Age       - 나이(0~  
 5   SibSp     - 형제자매+배우자 수  
 6   Parch     - 부모 + 자식 수  
 7   Ticket    - 티겟 번호  
 8   Fare      - 요금  
 9   Cabin     - 객실번호  
 10  Embarked  - 승선항  

In [16]:
# train , test 변수에 데이터 불러와서 담기
# 단, 인덱스를 승객의 번호로 설정해서 담아줄 것
train = pd.read_csv('data/titanic/train.csv', index_col = 'PassengerId')
test = pd.read_csv('data/titanic/test.csv', index_col = 'PassengerId')
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non

In [17]:
# 크기 확인
print('트레인 크기:', train.shape)
print('테스트 크기:', test.shape) # test 정답 데이터 x (-1)

트레인 크기: (891, 11)
테스트 크기: (418, 10)


In [18]:
# 컬럼명만 뽑아서 확인하기
print(train.columns)
print(test.columns)

# 컬럼의 순서도 중요함
# 컬럼의 명도 동일해야 함

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


### 데이터 전처리 및 데이터 탐색
#### 결측치 확인


In [25]:
# info 함수 통해서 train 결측치 확인
train.info()
# 결측치 존재 컬럼 Age, Cabin, Embarked

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [26]:
# info 함수 통해서 test 결측치 확인
test.info()
# 결측치 존재 컬럼 Age, Cabin

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [28]:
# pandas 함수 통해서 test 결측치 확인
# 데이터 자리에 결측치가 있니? -> 데이터.isnull()
test.isnull().sum()
# 결측치 존재 컬럼 : Age, Cabin, Fare
# 결측치(행)를 삭제하기 보다 채우면서 처리해보겠음

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

#### train-Embarked 결측치 처리
    - 승객이 탑승한 항구 이름
    

In [32]:
# train 데이터 기준으로 Embarked 컬럼 확인
# nan 결측치
train['Embarked'].unique()

# 빈도수가 높은 승선항 알파벳으로 결측치를 처리해보자
train['Embarked'].value_counts() # 대문자S 644로 채워보자!

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
# test 안보나요?
# 탐색용 데이터는 train , 
# 모델 학습 : 모델이 학습을 잘하도록 데이터를 세팅
# test 데이터는 모델을 일반화시키는데 도움이 되지 않음
# train 데이터가 학습할 때 사용되고 모델의 일반화에 도움을 줌

In [36]:
# Embarked 결측치를 'S'로 채우기
# fillna(채울값)
train['Embarked'] = train['Embarked'].fillna('S')
train['Embarked'].unique()
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
# 추가) Embarked 컬럼에 결측치가 있던 행의 정보를 확인하고
# 비슷한 사람의 데이터가 있다면 그 사람의 승선항 정보를 대체값으로 활용

#### test - Fare 결측치 처리
     - 요금 평균, 중앙값 계산 가능
     - 데이터 분석을 진행하면서 관련이 있을 법한 컬럼을 연관지어서 대체값을 확인해보자 

In [39]:
# train 기준으로 데이터 탐색
# Fare 컬럼 정보 확인 -> 기술통계량
train['Fare'].describe()
# 요금의 대표값으로 평균보다 중앙이 더 나을 수 있겠음

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [42]:
# 변수(특성) 간의 상관관계 확인
# 상관계수(숫자) : -1 ~ 1
# 각 컬럼의 상관 정도를 수치로 표현한 값 : 상관계수
# -1, 1에 가까울수록 변수간의 관계성이 있음
# 0에 가까울수록 변수간의 관계성이 적음
# 생존 사망과 연관이 있는 컬럼 확인해보기
train.corr()['Survived'].abs() 
# Pclass가 생존/사망 연관성이 있을 수 있겠다
# 등급에 따라 요금의 중앙값으로 채워주는 것도 의미가 있지 않을까? 

C:\Users\gjaischool\AppData\Local\Temp\ipykernel_20052\2439257001.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()['Survived'].abs()


Survived    1.000000
Pclass      0.338481
Age         0.077221
SibSp       0.035322
Parch       0.081629
Fare        0.257307
Name: Survived, dtype: float64

In [85]:
# 1. Pclass, Fare 두 개 컬럼만 인덱싱 해보기
# 2. Pclass 1,2,3 등급끼리 묶기(그룹화하기)
# 3. median() 집계함수 연결
# * 집계함수(count(), mean(), min(), max(), median())
train[['Pclass','Fare']].groupby('Pclass').median()

,Fare
Pclass,
1,60.2875
2,14.2500
3,8.0500
